In [ ]:
# Imports
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# Update with your API key
client = OpenAI(api_key=os.environ.get("API_KEY"))

# Open the CSV file in "read binary" (rb) mode, with the "assistants" purpose
file = client.files.create(
  file=open("Intro-to-Python-Part-1.pdf", "rb"),
  purpose='assistants'
)

# Create and configure the assistant
assistantResumer = client.beta.assistants.create(
    name="Teacher",
    instructions="""Please take the lecture PDFs and create detailed keypoints that are longer and more informative. The summary should include:
- All major topics covered in the lecture, with a concise explanation of each.
- Key concepts and definitions should be clearly stated and elaborated upon to ensure thorough understanding.
- Any complex academic content must be broken down into simpler terms for better grasp, while preserving the necessary technical terms to accurately convey the concepts.
- Examples or analogies that make the information relatable and easier to understand for an audience with varying levels of knowledge.
- Questions that commonly arise from the material, with straightforward answers to them.
- Maintain a warm and conversational tone throughout the text to keep it engaging.
The goal is to provide a full but accessible overview of the lecture content that is not just bullet points, but a rich narrative that educates and engages readers of all levels. Please ensure the summary is comprehensive and extends over several paragraphs, effectively translating dense material into an easy-to-follow format.""",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)

# Create a thread where the conversation will happen
thread = client.beta.threads.create()

# Create the user message and add it to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Here is my file:",
)

# Create the Run, passing in the thread and the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantResumer.id
)

# Periodically retrieve the Run to check status and see if it has completed
# Should print "in_progress" several times before completing
while run.status != "completed":
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    if keep_retrieving_run.status == "completed":
        break

# Retrieve messages added by the Assistant to the thread
all_messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Print the messages from the user and the assistant
print("###################################################### \n")
print(f"USER: {message.content[0].text.value}")
print(f"ASSISTANT: {all_messages.data[0].content[0].text.value}")

In [29]:
assistantQuizMaster = client.beta.assistants.create(
    name="Teacher",
    instructions="""You are tasked with crafting questions designed to aid students' learning and retention based on what they give you. You should generate a balanced mix of multiple-choice and short-answer questions. Each question should be concise, clear, and directly related to the provided content. Present your questions in JSON format and ensure the 'Answer' field is filled with the correct answer. Focus on maintaining a user-centered approach by adapting the question difficulty to individual study needs and promoting engagement.
### Example formating:
{
  "questions": [
    {
      "type": "multiple_choice",
      "question": "What substance in the cell membrane is primarily responsible for the facilitation of passive transport?",
      "options": [
        "a) Integral proteins",
        "b) Cholesterol",
        "c) Phospholipids",
        "d) Carbohydrates"
      ],
      "answer": "a) Integral proteins"
    },
    {
      "type": "short_answer",
      "question": "Describe the process of cell division in prokaryotic organisms.",
      "answer": "The process of cell division in prokaryotic organisms is known as binary fission. The process includes replication of the DNA, segregation of the copied chromosomes, and splitting of the parent cell into two daughter cells."
    }
  ]
}
""",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
)

# Create a thread where the conversation will happen
thread = client.beta.threads.create()

# Create the user message and add it to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f'{all_messages.data[0].content[0].text.value}',
)

# Create the Run, passing in the thread and the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantQuizMaster.id
)

# Periodically retrieve the Run to check status and see if it has completed
# Should print "in_progress" several times before completing
while run.status != "completed":
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

    if keep_retrieving_run.status == "completed":
        break

# Retrieve messages added by the Assistant to the thread
all_messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [30]:
print(all_messages.data[0].content[0].text.value)

{
  "questions": [
    {
      "type": "multiple_choice",
      "question": "What is the primary function of Parallel Virtual Machine (PVM) in the context of High Performance Computing (HPC)?",
      "options": [
        "a) To provide data storage solutions",
        "b) To support concurrent programming",
        "c) To network computers as a distributed parallel processor",
        "d) To enhance the graphical processing unit (GPU) performance"
      ],
      "answer": "c) To network computers as a distributed parallel processor"
    },
    {
      "type": "multiple_choice",
      "question": "What is a key feature of OpenNebula in cloud platform management?",
      "options": [
        "a) Multi-tenant cloud-like provisioning layer on top of existing infrastructure",
        "b) Hosting mobile applications",
        "c) Centralized database management system",
        "d) Integration with social media APIs"
      ],
      "answer": "a) Multi-tenant cloud-like provisioning layer on 

In [31]:
with open('questions.json', 'w') as f:
    f.write(all_messages.data[0].content[0].text.value)